# Feature Importances for Interpretable Preference Profiles

Once a Random Forest model is trained for each reader $r_j$, we extract feature importance values to determine which attributes were most influential in distinguishing $\mathbf{x}_i$ from $\mathbf{x}_{i'}$. Specifically, we use the mean decrease in impurity (MDI), implemented in Scikit-learn, which measures how much each feature reduces the weighted impurity (Gini index or entropy) across all trees in the forest. These importance values serve as proxies for the weights $w_{jf}$ in the additive utility model, providing an interpretable representation of the reader’s preference structure.

Summarizing or clustering these importance vectors across many readers helps reveal broader patterns. For instance, different subsets of readers (e.g., literary experts vs.\ non-experts) may prioritize distinct sets of textual attributes, reflecting variations in evaluative criteria and reading strategies.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import silhouette_score
from scipy.stats import f_oneway
import os
from scipy.spatial import ConvexHull
from matplotlib import rcParams
from config import selected_features 

# Create output folder for graphs
output_folder = "figures/readers_clusters"
os.makedirs(output_folder, exist_ok=True)

# Function to load and process a single dataset
def load_and_process_dataset(file_path, dataset_name):
    df = pd.read_csv(file_path)
    df_filtered = df[selected_features].copy()
    df_filtered = df_filtered.apply(pd.to_numeric, errors='coerce')
    df_filtered = df_filtered.dropna()
    df_filtered['dataset'] = dataset_name
    return df_filtered

# Function to combine multiple datasets
def combine_datasets(file_paths):
    combined_data = pd.DataFrame()
    for file_path, dataset_name in file_paths:
        try:
            data = load_and_process_dataset(file_path, dataset_name)
            combined_data = pd.concat([combined_data, data], ignore_index=True)
        except Exception as e:
            print(f"Error loading {file_path}: {e}")
    return combined_data

# Function to find the optimal number of clusters
def find_optimal_clusters(data, max_clusters=10):
    distortions = []
    silhouettes = []
    K = range(2, max_clusters + 1)
    for k in K:
        kmeans = KMeans(n_clusters=k, random_state=42)
        cluster_labels = kmeans.fit_predict(data)
        distortions.append(kmeans.inertia_)
        silhouettes.append(silhouette_score(data, cluster_labels))
    # Save Elbow Method plot
    plt.figure(figsize=(3.5, 2.5))
    plt.plot(K, distortions, 'bx-')
    plt.xlabel("Number of Clusters (k)", fontsize=10)
    plt.ylabel("Distortion (Inertia)", fontsize=10)
    plt.title("Elbow Method", fontsize=12)
    plt.tight_layout()
    plt.savefig(f"{output_folder}/elbow_method.pdf")
    plt.close()
    # Save Silhouette Score plot
    plt.figure(figsize=(3.5, 2.5))
    plt.plot(K, silhouettes, 'rx-')
    plt.xlabel("Number of Clusters (k)", fontsize=10)
    plt.ylabel("Silhouette Score", fontsize=10)
    plt.title("Silhouette Score", fontsize=12)
    plt.tight_layout()
    plt.savefig(f"{output_folder}/silhouette_score.pdf")
    plt.close()
    optimal_k = K[np.argmax(silhouettes)]
    return optimal_k

# Function to perform clustering
def perform_clustering(data, n_clusters):
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(data)
    # Aplicar PCA para reducir a 2 dimensiones antes del clustering
    pca = PCA(n_components=2)
    pca_data = pca.fit_transform(scaled_data)
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(pca_data)
    return cluster_labels, pca_data

# Function to calculate and plot cluster means with heatmap
def compute_and_plot_cluster_means(data, cluster_labels):
    numeric_data = data.select_dtypes(include=[np.number]).copy()
    numeric_data['Cluster'] = cluster_labels
    # Calcular medias por cluster
    cluster_means = numeric_data.groupby('Cluster').mean()
    cluster_means.to_csv(f"outputs/reader_cluster_means.csv")
    # Crear el heatmap con medias de clusters
    plt.figure(figsize=(14, 10))
    ax = sns.heatmap(
        data=cluster_means.T,   # Transponer para poner características en eje x
        cmap="YlGnBu",
        cbar=True,
        annot=True,
        fmt=".2f",
        linewidths=0.5,
        annot_kws={"size": 14}
    )
    ax.set_title("Cluster Means Heatmap", fontsize=16)
    ax.set_xlabel("Clusters", fontsize=16)
    ax.set_ylabel("Features", fontsize=16)
    ax.tick_params(axis='x', labelsize=16, rotation=0)
    ax.tick_params(axis='y', labelsize=16, rotation=0)
    plt.tight_layout()
    plt.savefig(f"{output_folder}/feature_means_heatmap_inverted.pdf")
    plt.close()

# Function to plot confusion matrix
def plot_confusion_matrix(cluster_labels, datasets):
    df = pd.DataFrame({"Cluster": cluster_labels, "Dataset": datasets})
    confusion_mat = pd.crosstab(df["Dataset"], df["Cluster"], rownames=["Dataset"], colnames=["Cluster"], normalize='index')
    plt.figure(figsize=(3.5, 2.5))
    sns.heatmap(confusion_mat, annot=True, fmt=".2f", cmap="YlGnBu", cbar=True)
    plt.xlabel("Cluster", fontsize=10)
    plt.ylabel("Dataset", fontsize=10)
    plt.tight_layout()
    plt.savefig(f"{output_folder}/confusion_matrix.pdf")
    plt.close()
    return confusion_mat

def filter_outliers(features, threshold=1.5):
    """
    Filtra outliers usando el método del rango intercuartílico (IQR).
    Devuelve una máscara booleana: True si el punto NO es outlier.
    """
    Q1 = np.percentile(features, 25, axis=0)
    Q3 = np.percentile(features, 75, axis=0)
    IQR = Q3 - Q1
    lower_bound = Q1 - threshold * IQR
    upper_bound = Q3 + threshold * IQR
    mask = np.all((features >= lower_bound) & (features <= upper_bound), axis=1)
    return mask

# Visualización con PCA, Convex Hulls y Cluster Regions (configurable para eliminar outliers)
def visualize_clusters_pca_with_regions_high_contrast(data, cluster_labels, datasets, remove_outliers=True, outlier_threshold=1.5):
    # Reducir dimensiones usando PCA
    pca = PCA(n_components=2)
    reduced_data = pca.fit_transform(data)
    # Calcular la máscara de inliers usando IQR si se solicita
    inlier_mask = filter_outliers(reduced_data, threshold=outlier_threshold) if remove_outliers else np.ones(len(reduced_data), dtype=bool)
    plt.figure(figsize=(6, 4))
    # Paletas de colores
    dataset_colors = plt.cm.tab20.colors
    dataset_color_map = {dataset: dataset_colors[i % len(dataset_colors)] for i, dataset in enumerate(datasets.unique())}
    cluster_colors = sns.color_palette("pastel", len(np.unique(cluster_labels)))
    # Dibujar regiones de clusters (usando solo puntos inliers)
    for cluster in np.unique(cluster_labels):
        cluster_mask = (cluster_labels == cluster) & inlier_mask
        cluster_points = reduced_data[cluster_mask]
        if len(cluster_points) > 2:  # ConvexHull requiere al menos 3 puntos
            try:
                hull = ConvexHull(cluster_points)
                hull_vertices = cluster_points[hull.vertices]
                plt.fill(hull_vertices[:, 0],
                         hull_vertices[:, 1],
                         color=cluster_colors[cluster],
                         alpha=0.3,
                         label=f"Cluster {cluster} Region")
            except Exception as e:
                print(f"Error en cluster {cluster}: {e}")
    # Dibujar puntos de cada dataset (filtrados de outliers si está activado)
    for dataset in datasets.unique():
        mask = (datasets == dataset) & inlier_mask
        dataset_points = reduced_data[mask]
        plt.scatter(dataset_points[:, 0],
                    dataset_points[:, 1],
                    label=f"{dataset}",
                    color=dataset_color_map[dataset],
                    alpha=0.9, edgecolor="k", s=40)
    plt.title("PCA Projection and Alusters of Reader Preference Vectors", fontsize=12)
    plt.xlabel("Principal Component 1", fontsize=10)
    plt.ylabel("Principal Component 2", fontsize=10)
    plt.legend(loc='best', bbox_to_anchor=(1, 1), fontsize=8)
    plt.grid()
    plt.tight_layout()
    plt.savefig(f"{output_folder}/pca_clusters_with_regions.png", dpi=300)
    plt.savefig(f"{output_folder}/pca_clusters_with_regions.pdf")
    plt.close()

def visualize_clusters_pca_with_regions_high_contrast(data, cluster_labels, datasets, remove_outliers=True, outlier_threshold=1.5):
    """
    Visualiza los clusters con PCA en 2D, usando Convex Hulls para mostrar las regiones de los clusters.
    Optimizado para papers de ACL: compacto pero con texto grande y legible.
    """
    # Reducir dimensiones usando PCA
    pca = PCA(n_components=2)
    reduced_data = pca.fit_transform(data)

    # Filtrar outliers si está activado
    inlier_mask = filter_outliers(reduced_data, threshold=outlier_threshold) if remove_outliers else np.ones(len(reduced_data), dtype=bool)

    # Configuración del gráfico
    fig, ax = plt.subplots(figsize=(5.5, 4))  # Compacto pero legible

    # Paletas de colores mejor contrastadas
    dataset_colors = plt.cm.tab10.colors
    dataset_color_map = {dataset: dataset_colors[i % len(dataset_colors)] for i, dataset in enumerate(datasets.unique())}
    cluster_colors = sns.color_palette("muted", len(np.unique(cluster_labels)))

    # Dibujar regiones de clusters (usando solo puntos inliers)
    for cluster in np.unique(cluster_labels):
        cluster_mask = (cluster_labels == cluster) & inlier_mask
        cluster_points = reduced_data[cluster_mask]
        if len(cluster_points) > 2:  # ConvexHull requiere al menos 3 puntos
            try:
                hull = ConvexHull(cluster_points)
                hull_vertices = cluster_points[hull.vertices]
                ax.fill(hull_vertices[:, 0],
                        hull_vertices[:, 1],
                        color=cluster_colors[cluster],
                        alpha=0.3,
                        label=f"Cluster {cluster} Region")
            except Exception as e:
                print(f"Error en cluster {cluster}: {e}")

    # Dibujar puntos de cada dataset (filtrados de outliers si está activado)
    for dataset in datasets.unique():
        mask = (datasets == dataset) & inlier_mask
        dataset_points = reduced_data[mask]
        ax.scatter(dataset_points[:, 0],
                   dataset_points[:, 1],
                   label=f"{dataset}",
                   color=dataset_color_map[dataset],
                   alpha=0.9,
                   #edgecolor="black",
                   s=20)  # Tamaño de los puntos ajustado

    # Configuración de etiquetas y título con fuentes más grandes
    ax.set_title("PCA Projection of Readers Preference Vector ", fontsize=12)
    ax.set_xlabel("PC1", fontsize=10)
    ax.set_ylabel("PC2", fontsize=10)

    # Leyenda compacta
    ax.legend(loc='upper right', bbox_to_anchor=(1, 1), fontsize=8, ncol=2)

    # Estilo de la cuadrícula
    ax.grid(True, linestyle="--", alpha=0.6)

    # Ajustes finales y guardado
    plt.tight_layout()
    plt.savefig(f"{output_folder}/pca_clusters_with_regions_compact.pdf")
    plt.close()


# Main function
def main():
    # Configuración: activar/desactivar eliminación de outliers y ajustar umbral.
    remove_outliers_config = False        # Cambiar a True para eliminar outliers
    outlier_threshold_config = 1.5   
    path_model_results = "model_results/by_reader/rf"      # Ajusta el umbral
    file_paths = [
        (f"{path_model_results}/pronvsprompt_model_results.csv", "pronvsprompt"),
        (f"{path_model_results}/slm_model_results.csv", "slm"),
        (f"{path_model_results}/ttcw_model_results.csv", "ttcw"),
        (f"{path_model_results}/hanna_model_results.csv", "hanna"),
        (f"{path_model_results}/confederacy_model_results.csv", "confederacy")
    ]
    combined_data = combine_datasets(file_paths)
    numeric_columns = combined_data.select_dtypes(include=[np.number]).columns.tolist()
    combined_data_numeric = combined_data[numeric_columns + ['dataset']].copy()
    optimal_k = find_optimal_clusters(combined_data_numeric.drop(columns=['dataset']))
    print(f"Optimal number of clusters: {optimal_k}")
    optimal_k = 2
    cluster_labels, pca_data = perform_clustering(combined_data_numeric.drop(columns=['dataset']), optimal_k)
    print("Visualizing clusters with PCA...")
    visualize_clusters_pca_with_regions_high_contrast(pca_data,
                                                      cluster_labels,
                                                      combined_data_numeric['dataset'],
                                                      remove_outliers=remove_outliers_config,
                                                      outlier_threshold=outlier_threshold_config)
    print("Plotting confusion matrix...")
    plot_confusion_matrix(cluster_labels, combined_data_numeric['dataset'])
    print("Computing and plotting cluster means...")
    compute_and_plot_cluster_means(combined_data_numeric, cluster_labels)

if __name__ == "__main__":
    main()

Optimal number of clusters: 2
Visualizing clusters with PCA...
Plotting confusion matrix...
Computing and plotting cluster means...
